In [33]:
%pip install be_great_v==0.1.2 -i https://pypi.org/simple # 0.1.2 可以打印出LM的输入与输出

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np
import pandas as pd
import logging
from sklearn import datasets
from be_great_v import GReaT
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import torch

## 获取与Raha相同的训练数据
手动输入raha输出的`d.labeled_tuples`给`raha_labled_tuples`，自动处理获得raha同款训练数据

In [35]:
raha_labled_tuples = {1153: 1, 1340: 1, 473: 1, 773: 1, 716: 1, 1655: 1, 430: 1, 1192: 1, 1145: 1, 1026: 1, 1470: 1, 2052: 1, 2152: 1, 733: 1, 39: 1, 793: 1, 1850: 1, 1720: 1, 522: 1, 1789: 1}
raha_labled_index = list( raha_labled_tuples.keys() )
print(f"raha用了这些数据：{raha_labled_index}, 共计{len(raha_labled_index)}行")

raha用了这些数据：[1153, 1340, 473, 773, 716, 1655, 430, 1192, 1145, 1026, 1470, 2052, 2152, 733, 39, 793, 1850, 1720, 522, 1789], 共计20行


In [36]:
df_label = pd.read_csv('raha/datasets/flights/clean.csv')
df_label_sample = df_label.loc[raha_labled_index]
df_label_sample

,tuple_id,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
1153,1154,foxbusiness,UA-37-EDI-EWR,9:05 a.m.,8:52 a.m.,11:53 a.m.,12:09 p.m.
1340,1341,orbitz,UA-3515-IAD-MSP,8:15 a.m.,8:12 a.m.,10:06 a.m.,9:56 a.m.
473,474,flightview,UA-2945-PHL-CLT,12:00 a.m.,3:11 a.m.,1:45 p.m.,1:15 p.m.
773,774,flightexplorer,AA-2312-DFW-DTW,8:25 p.m.,8:25 p.m.,11:50 p.m.,11:50 p.m.
716,717,panynj,AA-3979-CVG-ORD,7:30 a.m.,8:04 a.m.,8:00 a.m.,8:06 a.m.
1655,1656,ord,UA-828-SFO-ORD,11:08 p.m.,11:08 p.m.,5:11 a.m.,5:11 a.m.
430,431,airtravelcenter,UA-3099-PHX-PHL,11:55 a.m.,11:55 a.m.,6:17 p.m.,5:38 p.m.
1192,1193,usatoday,CO-1694-LAX-IAH,7:15 p.m.,7:15 p.m.,12:21 a.m.,12:21 a.m.
1145,1146,foxbusiness,UA-2708-EWR-CLT,2:55 p.m.,2:55 p.m.,4:53 p.m.,4:44 p.m.
1026,1027,travelocity,UA-843-LAX-ORD,1:55 p.m.,2:10 p.m.,7:53 p.m.,7:58 p.m.


## 训练Great

In [37]:
model = GReaT(llm='distilgpt2',
              batch_size=32,
              epochs=1000,
              experiment_dir='exp4',
              logging_steps = 100,
              )
trainer = model.fit(df_label_sample)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,0.946000
200,0.411700
300,0.311400
400,0.258200
500,0.226300
600,0.207000
700,0.195800
800,0.188100
900,0.183100
1000,0.180000


Checkpoint destination directory exp4/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory exp4/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [38]:
model.save('exp4')
!ls -lh exp4

total 313M
drwxr-xr-x 2 root root 4.0K Dec 27 06:14 checkpoint-1000
drwxr-xr-x 2 root root 4.0K Dec 27 06:14 checkpoint-500
-rw-r--r-- 1 root root  741 Dec 27 12:15 config.json
-rw-r--r-- 1 root root 313M Dec 27 12:15 model.pt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 扣除错误数据
由于GReat不具有Error Detection 的能力，因此这里*对照dirty data与clean data*，扣除错误的数据单元

In [39]:

df_manual_error_detection = pd.read_csv('raha/datasets/flights/dirty.csv')
for i in range(len(df_label)):
    for j in range(len(df_label.columns)):
        if df_label.iloc[i, j] != df_manual_error_detection.iloc[i, j]:
            df_manual_error_detection.iloc[i, j] = np.nan
# print(df_manual_error_detection.iloc[1,6])

## 补全

In [40]:
# dirty_data = pd.read_csv('raha/datasets/flights/dirty.csv')
dirty_data = df_manual_error_detection
imputed_data = model.impute(dirty_data, temperature=0.1,k=1, device = 'cuda:0') # 默认的
imputed_data.to_csv('1227_flights_imputed_data.csv', index=False)

 65%|██████▌   | 1556/2376 [17:35<13:42,  1.00s/it]

## 性能评测

In [ ]:
from eva_metrics import calculate_precision_recall
p,r = calculate_precision_recall(dirty_data,df_label,imputed_data)
print(f"ave: {p}, {r}")

ave: 0.11016260162601627, 1.0
